In [302]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
#import seaborn as sns
%matplotlib inline
#import re
#import time
#from datetime import datetime
#import matplotlib.dates as mdates
#import matplotlib.ticker as ticker
#from urllib.request import urlopen
import requests

# ECON 323 FINAL PROJECT: ANALYSIS CODE

## 1. Data Cleaning
The data that a scraped is not anywhere close to clean so there is a significant amount of work to do to make it useful.

### 1.1 Import Data
First I import all 5 datasets outputted from the scraping code and merge them into one data frame.

In [303]:
all_files = glob.glob( "scraped_data/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

raw_amazon_data = pd.concat(li, axis=0, ignore_index=True)

raw_amazon_data.to_csv('raw_amazon_data.csv', index=False, encoding='utf-8')

raw_amazon_data.head()

,Book Name,Author,Price,KU,Rankings,Rating,Customers_Rated,Blurb,Reviews_Ratings,Reviews_Titles,Reviews_Texts
0,TAGGED BY DEATH (RILEY MALLOY THRILLER SERIES...,['Judith A. Barrett'],CDN$ 6.99,1,"Best Sellers Rank: #6,162 in Kindle Store (...",4.0 out of 5 stars,71 ratings,\n Only the dog whisperer can identify a seri...,[],[],[]
1,The Armstrong Assignment (A Janet Markham Ben...,['Diana Xarissa'],CDN$ 3.99,1,"Best Sellers Rank: #1,736 in Kindle Store (...",4.6 out of 5 stars,316 ratings,\n Janet Markham’s wedding day is perfect in ...,"['5.0 out of 5 stars', '5.0 out of 5 stars']","['\nA New Adventure\n', '\nA lovely page turne...",['\n\n Janet and Edward are starting their li...
2,Overachiever (Slumming It Book 2),['S.M. Shade'],CDN$ 5.10,1,"Best Sellers Rank: #3,630 in Kindle Store (...",4.7 out of 5 stars,107 ratings,"\n ""Overachiever was hands down one of the be...","['5.0 out of 5 stars', '4.0 out of 5 stars', '...","['\nWow\n', '\nA laugh-out-loud college romanc...",['\n\n I cannot wait for the 3rd book in this...
3,The Perfect Couple: A gripping USA Today psyc...,['Jackie Kabler'],CDN$ 5.99,1,Best Sellers Rank: #1 in Kindle Store (See ...,4.1 out of 5 stars,"11,783 ratings",\n THE 2020 BESTSELLER!Perfect for fans of Li...,"['4.0 out of 5 stars', '2.0 out of 5 stars', '...","['\nsolid, kept me guessing\n', '\nTypical pre...","[""\n\n This is a plot driven story and it wor..."
4,Sold on a Monday: A Novel,['Kristina McMorris'],CDN$ 11.10,1,Best Sellers Rank: #2 in Kindle Store (See ...,4.2 out of 5 stars,"15,798 ratings",\n A NEW YORK TIMES BESTSELLER A USA TODAY BE...,"['5.0 out of 5 stars', '5.0 out of 5 stars', '...","['\nTruth bares fiction\n', '\nLoved it\n', '\...",['\n\n It is a bit of the reality of the time...


### 1.2 Clean Numeric Variables


In [397]:
df = pd.read_csv('raw_amazon_data.csv', index_col=None, header=0)

#define clean string function
def clean_str(x, r_text):
    if isinstance(x, str):
        return(x.replace(r_text, ''))
    return(x)

#convert price to float
df['Price'] = raw_amazon_data['Price'].apply(clean_str, args =("CDN$ ",)).astype(float)

#separate kindle store overall rankings from other rankings, where no overall kindle stroe ranking, set NaN
df['Rankings'] = df['Rankings'].astype(str)
df['Rankings'] = np.where(df['Rankings'].str.contains(" in Kindle Store", na=False),df['Rankings'], "NO KS RANK")
df['Rankings'] = df['Rankings'].apply(lambda x: x.split('#',1)[-1])
df['Rankings'] = df['Rankings'].apply(lambda x: x.split(' in Kindle Store')[0])

#convert ranking to float
df['Rankings'] = df['Rankings'].str.replace(",", "")
df['Rankings'] = df['Rankings'].replace("NO KS RANK", np.nan)
df['Rankings'] = df['Rankings'].replace("37 Free", np.nan)
df['Rankings'] = df['Rankings'].astype(float)

#convert rating to float
df['Rating'] = df['Rating'].apply(clean_str, args =(" out of 5 stars",)).astype(float)

#convert customers_rated to float
df['Customers_Rated'] = df['Customers_Rated'].apply(clean_str, args =(" ratings",))
df['Customers_Rated'] = df['Customers_Rated'].apply(clean_str, args =(" rating",))
df['Customers_Rated'] = df['Customers_Rated'].apply(clean_str, args =("[]",))
df['Customers_Rated'] = df['Customers_Rated'].replace("", np.nan)
df['Customers_Rated'] = df['Customers_Rated'].apply(clean_str, args =(",",)).astype(float)

#df.head()

### 1.3 Clean String Variables

In [398]:
#define function to format author(s)
def author_list(x):
    a = ''
    for i in range(0, len(x)):
        if i == 0:
            a = str(x[i])
        elif i != len(x) - 1:
            a += (" , " + str(x[i]))
        else:
            a += (" & " + str(x[i]))
    return a

#reformat author variable and convert to string
df['Author'] = df['Author'].apply(eval)
df['Author'] = df['Author'].apply(author_list).astype(str)

#reformat blurb variable
df['Blurb'] = df['Blurb'].apply(clean_str, args =("\n",)).astype(str)

### 1.4 Clean Review Variables

In [399]:
df['Reviews_Ratings'] = df['Reviews_Ratings'].apply(eval)

df_reviews = df['Reviews_Ratings'].apply(pd.Series)
df_reviews = df_reviews.applymap(lambda x: clean_str(x, " out of 5 stars"))
df_reviews = df_reviews.astype(float)
df_reviews["Average_Review_Rating"] = df_reviews.mean(axis = 1, skipna=True)

df["Average_Review_Rating"] = df_reviews["Average_Review_Rating"]

In [401]:
df['Reviews_Titles'] = df['Reviews_Titles'].replace(np.nan, "[]")
df['Reviews_Titles'] = df['Reviews_Titles'].astype(str)
df['Reviews_Titles'] = df['Reviews_Titles'].apply(eval)

df_reviews_title = df['Reviews_Titles'].apply(pd.Series)
df_reviews_title = df_reviews_title.applymap(lambda x: clean_str(x, "\n"))
df_reviews_title.columns=["Review_1_Title", "Review_2_Title", "Review_3_Title", "Review_4_Title", "Review_5_Title", "Review_6_Title", "Review_7_Title","Review_8_Title"]
df_reviews_title = df_reviews_title.replace(np.nan, "")
df_reviews_title['All_Review_Titles'] = df_reviews_title.agg(' '.join, axis=1).str.strip()
df_reviews_title = df_reviews_title.replace("", np.nan)
df = pd.concat([df,df_reviews_title], axis=1)

In [402]:
df['Reviews_Texts'] = df['Reviews_Texts'].replace(np.nan, "[]")
df['Reviews_Texts'] = df['Reviews_Texts'].astype(str)
df['Reviews_Texts'] = df['Reviews_Texts'].apply(eval)

df_reviews_text = df['Reviews_Texts'].apply(pd.Series)
df_reviews_text = df_reviews_text.applymap(lambda x: clean_str(x, "\n"))
df_reviews_text.columns=["Review_1", "Review_2", "Review_3", "Review_4", "Review_5", "Review_6", "Review_7","Review_8"]
df_reviews_text = df_reviews_text.replace(np.nan, "")
df_reviews_text['All_Review_Text'] = df_reviews_text.agg(' '.join, axis=1).str.strip()
df_reviews_text = df_reviews_text.replace("", np.nan)
df = pd.concat([df,df_reviews_text], axis=1)

In [407]:
df.drop(['Reviews_Ratings', 'Reviews_Texts', 'Reviews_Titles'], axis=1, inplace=True)

In [409]:
df.to_csv('clean_amazon_data.csv', index=False, encoding='utf-8')

In [3]:
r = requests.post(
    "https://api.deepai.org/api/sentiment-analysis",
    data={
        'text': 'I HATE THIS',
    },
    headers={'api-key': 'quickstart-QUdJIGlzIGNvbWluZy4uLi4K'}
)
print(r.json())

{'status': "Looks like you're enjoying our API. Want to keep using it? Sign up to get an API Key that's as unique as you are. It's free! https://deepai.org/"}
